In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
from sklearn.metrics import confusion_matrix
import itertools
import re
import os
import pickle
from datetime import datetime

In [129]:
df_page0 = pd.read_csv('test_page.csv', dtype='object')
df0 = pd.read_csv('test_event.csv', dtype='object')

In [132]:
df_page = df_page.copy()
df = df.copy()

In [40]:
copy = df_page.copy()
copy2 = copy.sort_values('hitNumber', ascending=False).drop_duplicates(subset = 'fullVisitorId', keep = 'last')
#copy2 = copy.sort_values('hitNumber', ascending=False)
copy2.fillna(0, inplace=True)

In [41]:
copy2.drop(columns=['thankyou','dhealth_suc_s2','timeOnSite','health_yesno','prodgroup','prodgroup2','hitNumber','session_health','pathlastlv2',
                    'dhealth_thankyou','category','action','label','product_health','pathlastlv','pathlv1','pathlv2',
                    'pathlv3','pathlv4','visitId','date','hits_type','pagePath','nextTime','prevTime','timeOnPage','source','medium','campaign'],inplace=True)

In [42]:
train=copy2.sample(frac=0.8,random_state=200) #random state is a seed value
test=copy2.drop(train.index)

In [1]:
df_dummy = pd.get_dummies(train[['os','channelGrouping','source_new','medium_new']])

train_new = pd.concat([train, df_dummy], axis=1)
train_new.drop(columns=['os','channelGrouping','source_new','medium_new'],inplace=True)


In [2]:
clf = tree.DecisionTreeClassifier()
X = train_new.copy()
X = X.drop(columns=['fullVisitorId','dhealth_suc_u'])
X = X.values.astype(float)
y = train_new['dhealth_suc_u'].values.astype(int)

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#clf = XGBClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

In [154]:
pd.options.display.float_format = '{:,.2f}'.format

importance = pd.DataFrame({'Feature':train_new.drop(columns=['fullVisitorId','dhealth_suc_u']).columns.values, 'Importance':clf.feature_importances_})
importance.sort_values(by=['Importance'],ascending=[False],inplace=True)
print(importance[0:10])

# Confusion matrix 
y_pred = clf.predict(X_test)

print('')

print("Test Accuracy: ",round(accuracy_score(y_test,y_pred),2))
print("Test F1 Score: ",round(f1_score(y_test,y_pred),2))
print('')
print('')
print("Confusion Matrix on Test Data")
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

                Feature  Importance
2      page_per_session        0.23
8         page_per_user        0.20
7              prodview        0.13
1         session_visit        0.08
0             day_visit        0.07
17               os_iOS        0.05
10           os_Android        0.04
26  source_new_facebook        0.03
16           os_Windows        0.03
35       medium_new_sem        0.03

Test Accuracy:  0.99
Test F1 Score:  0.04


Confusion Matrix on Test Data


Predicted,0,1
Actual,,
0,18680,42
1,104,3
